# Обратная задача ЭЭГ

Обратная задача ЭЭГ заключается в восстановлении распределения источников (тока и заряда) внутри области (например, головы) на основе измерений электрического потенциала на границе.

В данном ноутбуке мы используем нейросеть для моделирования распределения заряда и тока, а также обновляем функцию потерь и пайплайн для решения этой задачи.

In [1]:
# Импорт необходимых библиотек
using NeuralPDE, Lux, ModelingToolkit, Optimization, OptimizationOptimJL, LineSearches,
      OptimizationOptimisers, LuxCUDA, Random, ComponentArrays
using ModelingToolkit: Interval, infimum, supremum
using Distributions, Plots, CUDA

In [2]:
const gpud = gpu_device()
const cpud = cpu_device()

(::CPUDevice) (generic function with 4 methods)

In [ ]:
# Определение прямой задачи
@parameters x, y, z, t
@variables φ(..), Ax(..),Ay(..),Az(..), ρ(..), jx(..), jy(..), jz(..)
A = [Ax, Ay, Az]
j = [jx, jy, jz]
Dxx = Differential(x)^2
Dyy = Differential(y)^2
Dzz = Differential(z)^2

# Определение физических постоянных
const c = 2.99792458e10 # Скорость света в вакууме (см/с)
const ε₀ = 1.0 # Диэлектрическая постоянная вакуума в СГС (размерность отсутствует)
const ε = 1.0  # Диэлектрическая проницаемость (отн.)
const μ₀ = 1.0 # Магнитная постоянная вакуума в СГС (размерность отсутствует)
const μ = 1.0  # Магнитная проницаемость (отн.)


# Определение оператора Лапласа как функции
function laplacian(F, params)
    return sum((Differential(param)^2)(F) for param in params)
end

# Определение оператора Даламбера как функции
function dalembert_operator(F, params, ε, μ, c)
    Δ = laplacian(F, params)
    return Δ  - (ε * μ / c^2) * (Differential(t)^2)(F)
end


dalembert_operator (generic function with 1 method)

In [4]:
dalembert_operator(φ(x,y,z,t), [x, y, z], ε, μ, c)

Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t)))

In [5]:
# Уравнение
eqs = [

    [dalembert_operator(φ(x, y, z, t), [x, y, z], ε, μ, c) ~ -4 * pi * ρ(x, y, z, t) / ε];
    [dalembert_operator(A[i](x, y, z, t), [x, y, z], ε, μ, c) ~ -μ * 4 * pi / c* j[i](x, y, z, t) for i in 1:3];
    (Differential(x)(Ax(x, y, z, t)) + Differential(y)(Ay(x, y, z, t)) + Differential(z)(Az(x, y, z, t)) + (ε * μ / c) * Differential(t)(φ(x, y, z, t))) ~ 0.0
]

5-element Vector{Equation}:
 Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t))) ~ -12.566370614359172ρ(x, y, z, t)
 Differential(x)(Differential(x)(Ax(x, y, z, t))) + Differential(z)(Differential(z)(Ax(x, y, z, t))) + Differential(y)(Differential(y)(Ax(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ax(x, y, z, t))) ~ -4.191690043903363e-10jx(x, y, z, t)
 Differential(y)(Differential(y)(Ay(x, y, z, t))) + Differential(z)(Differential(z)(Ay(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ay(x, y, z, t))) + Differential(x)(Differential(x)(Ay(x, y, z, t))) ~ -4.191690043903363e-10jy(x, y, z, t)
 Differential(z)(Differential(z)(Az(x, y, z, t))) + Differential(y)(Differential(y)(Az(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Az(x, y, z, t))) + Differential

In [6]:
# Границы области
const x_min, x_max = -10.0, 10.0
const y_min, y_max = -10.0, 10.0
const z_min, z_max = -10.0, 10.0
const t_min, t_max = 0.0, 1.0
# Область
domains = [x ∈ Interval(x_min, x_max),
           y ∈ Interval(y_min, y_max),
           z ∈ Interval(z_min, z_max), 
           t ∈ Interval(t_min, t_max)]
# Граничные условия
measured_points = [
    [0.5, 0.5, 0.5, 0., 1.0],
    [0.2, 0.3, 0.4, 0., 0.0],
    [0.8, 0.7, 0.6, 0.,  -1.0]
] |> gpud

bcs = [
    [φ(x_min, y, z, t) ~ 0.0, φ(x_max, y, z, t) ~ 0.0,
    φ(x, y_min, z, t) ~ 0.0, φ(x, y_max, z, t) ~ 0.0,
    φ(x, y, z_min, t) ~ 0.0, φ(x, y, z_max, t) ~ 0.0];
    [A[i](x_min, y, z, t)  ~ 0.0 for i in 1:3]
]


9-element Vector{Equation}:
 φ(-10.0, y, z, t) ~ 0.0
 φ(10.0, y, z, t) ~ 0.0
 φ(x, -10.0, z, t) ~ 0.0
 φ(x, 10.0, z, t) ~ 0.0
 φ(x, y, -10.0, t) ~ 0.0
 φ(x, y, 10.0, t) ~ 0.0
 Ax(-10.0, y, z, t) ~ 0.0
 Ay(-10.0, y, z, t) ~ 0.0
 Az(-10.0, y, z, t) ~ 0.0

In [16]:
function additional_loss_weightened(lambda)
    
    function additional_loss(phi_pred_fun, θ, p_)
        CUDA.allowscalar() do
            # phi is first output of phi_pred_fun
            result = sum(abs2(phi_pred_fun([x, y, z, t]|>cpud, θ|>cpud)[1] - phi|>cpud) for (x, y, z, t, phi) in measured_points) / length(measured_points)|>cpud
            result = result * lambda
            return result
        end
    end
    
    return additional_loss
end

additional_loss_weightened (generic function with 1 method)

In [42]:

# Нейросеть
# Определение новой нейросети для моделирования распределения заряда и тока
input_ = 4  # x, y, z
n = 32      # число нейронов в скрытых слоях
lambda = 10 # вес дополнительной потерь
# Функция для разделения выхода сети на переменные
"""
function split_outputs(out)
    φ_pred = out[1]
    A_pred = out[2:4]
    ρ_pred = out[5]
    j_pred = out[6:8]
    return φ_pred, A_pred, ρ_pred, j_pred
end

chain = [Chain(
    Dense(input_, n, σ),
    Dense(n, n, σ),
    Dense(n, 1)
) for _ in 1:8] 

# Определение системы
ps = [Lux.setup(Random.default_rng(), chain[i])[1] |> ComponentArray |> gpud .|> Float64 for i in 1:8]
"""
chain = Chain(
    Dense(input_, n, σ),
    Dense(n, n, σ),
    Dense(n, n, σ),
    Dense(n, 8)
)

# Определение системы
ps = Lux.setup(Random.default_rng(), chain)[1] |> ComponentArray |> gpud .|> Float32

strategy = QuasiRandomTraining(4096)
discretization = PhysicsInformedNN(chain, strategy; init_params = ps, additional_loss = additional_loss_weightened(lambda), 
log_options = LogOptions(; log_frequency = 50))

PhysicsInformedNN{Chain{@NamedTuple{layer_1::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_2::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_3::Dense{typeof(σ), Int64, Int64, Nothing, Nothing, Static.True}, layer_4::Dense{typeof(identity), Int64, Int64, Nothing, Nothing, Static.True}}, Nothing}, QuasiRandomTraining{QuasiMonteCarlo.LatinHypercubeSample}, ComponentVector{Float32, CuArray{Float32, 1, CUDA.DeviceMemory}, Tuple{Axis{(layer_1 = ViewAxis(1:160, Axis(weight = ViewAxis(1:128, ShapedAxis((32, 4))), bias = ViewAxis(129:160, Shaped1DAxis((32,))))), layer_2 = ViewAxis(161:1216, Axis(weight = ViewAxis(1:1024, ShapedAxis((32, 32))), bias = ViewAxis(1025:1056, Shaped1DAxis((32,))))), layer_3 = ViewAxis(1217:2272, Axis(weight = ViewAxis(1:1024, ShapedAxis((32, 32))), bias = ViewAxis(1025:1056, Shaped1DAxis((32,))))), layer_4 = ViewAxis(2273:2536, Axis(weight = ViewAxis(1:256, ShapedAxis((8, 32))), bias = ViewAxis(257:264, Shaped1DAxis((8,)))

In [43]:
additional_loss_weightened(lambda)(discretization.phi, ps, 0)

6.724122f0

In [45]:
allvars = [φ(x, y, z, t); [A_(x, y, z, t) for A_ in A]; ρ(x, y, z, t); [j_(x, y, z, t) for j_ in j]]

8-element Vector{Num}:
  φ(x, y, z, t)
 Ax(x, y, z, t)
 Ay(x, y, z, t)
 Az(x, y, z, t)
  ρ(x, y, z, t)
 jx(x, y, z, t)
 jy(x, y, z, t)
 jz(x, y, z, t)

In [46]:
@named pde_system = PDESystem(eqs, bcs, domains, [x, y, z, t], allvars)

PDESystem
Equations: Equation[Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t))) ~ -12.566370614359172ρ(x, y, z, t), Differential(x)(Differential(x)(Ax(x, y, z, t))) + Differential(z)(Differential(z)(Ax(x, y, z, t))) + Differential(y)(Differential(y)(Ax(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ax(x, y, z, t))) ~ -4.191690043903363e-10jx(x, y, z, t), Differential(y)(Differential(y)(Ay(x, y, z, t))) + Differential(z)(Differential(z)(Ay(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ay(x, y, z, t))) + Differential(x)(Differential(x)(Ay(x, y, z, t))) ~ -4.191690043903363e-10jy(x, y, z, t), Differential(z)(Differential(z)(Az(x, y, z, t))) + Differential(y)(Differential(y)(Az(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Az(x, y, z, t))) + Differentia

In [47]:
prob = discretize(pde_system, discretization)
sym_prob = symbolic_discretize(pde_system, discretization)


NeuralPDE.PINNRepresentation(Equation[Differential(y)(Differential(y)(φ(x, y, z, t))) + Differential(x)(Differential(x)(φ(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(φ(x, y, z, t))) + Differential(z)(Differential(z)(φ(x, y, z, t))) ~ -12.566370614359172ρ(x, y, z, t), Differential(x)(Differential(x)(Ax(x, y, z, t))) + Differential(z)(Differential(z)(Ax(x, y, z, t))) + Differential(y)(Differential(y)(Ax(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ax(x, y, z, t))) ~ -4.191690043903363e-10jx(x, y, z, t), Differential(y)(Differential(y)(Ay(x, y, z, t))) + Differential(z)(Differential(z)(Ay(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Ay(x, y, z, t))) + Differential(x)(Differential(x)(Ay(x, y, z, t))) ~ -4.191690043903363e-10jy(x, y, z, t), Differential(z)(Differential(z)(Az(x, y, z, t))) + Differential(y)(Differential(y)(Az(x, y, z, t))) - 1.1126500560536184e-21Differential(t)(Differential(t)(Az(x, y, z, t))) + Dif

In [48]:
#phi = discretization.phi
pde_inner_loss_functions = sym_prob.loss_functions.pde_loss_functions
bcs_inner_loss_functions = sym_prob.loss_functions.bc_loss_functions
add_functon = sym_prob.loss_functions.additional_loss_function

callback = function (p, l)
    println("loss: ", l)
    println("pde_losses: ", map(l_ -> l_(p.u), pde_inner_loss_functions))
    println("bcs_losses: ", map(l_ -> l_(p.u), bcs_inner_loss_functions))
    #println("add_losses: ", add_functon(phi, p.u, nothing))
    return false
end

#61 (generic function with 1 method)

In [49]:
# Оптимизация
opt = OptimizationOptimisers.Adam(0.001)

Adam(0.001, (0.9, 0.999), 1.0e-8)

In [51]:

# Решение
res = solve(prob, opt; maxiters = 1000, callback)
phi = discretization.phi

loss: 38.71272186327799
pde_losses: [30.257044526670015, 4.488898536942605e-6, 4.525494186495245e-6, 4.541320244420278e-6, 1.7204086631341125e-5]
bcs_losses: [0.19426585567675325, 0.19010248338111632, 0.19236375744309453, 0.19052688046907262, 0.18819478881790258, 0.19493481262338533, 0.19434197013085364, 0.1944177787850357, 0.19434012383799976]
loss: 35.61822437059826
pde_losses: [27.35662294363814, 4.510440868964366e-6, 4.541956006603594e-6, 4.526579326713097e-6, 1.6995265543571292e-5]
bcs_losses: [0.17542275790081507, 0.17212079823594567, 0.17355377605130445, 0.17259695671529332, 0.16990134278895805, 0.17647403369631798, 0.17543166252719056, 0.17543372736267912, 0.1753767076339468]
loss: 32.738905004951086
pde_losses: [24.64015497229812, 4.484172176654854e-6, 4.456514820701584e-6, 4.537004137584449e-6, 1.630867512991233e-5]
bcs_losses: [0.15782315263811403, 0.15534448340087223, 0.15597477393483528, 0.1558570673484582, 0.15289451718831235, 0.15928677636973282, 0.1578302329493939, 0.15

LoadError: InterruptException: